In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.36 and you have 0.31 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/fv_builder/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 4
%connections dev-glue-redshift-connection
%idle_timeout 60

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/fv_builder/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 4
Connections to be included:
dev-glue-redshift-connection
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 4
Session ID: d5fcc2d4-b43d-4fdb-acc6-4f710f708a4b
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/fv_builder/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session d5fcc2d4-b43d-4fdb-acc6-4f710f708a4b to get into ready status...
Session d5fcc2d4-b43d-4fdb-acc6-4f710f708a

In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [6]:
from sstm_transformation.tsm_builder import TSMBuilder
from pyspark.sql.functions import input_file_name
import pyspark.sql.functions as F



In [ ]:
df_projectphaes = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/phases/*.parquet")

In [ ]:
df_projectphaes.select("phase").distinct().collect()

In [ ]:
base_path = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/fv_raw_data"

writeToFile(df_projectphaes, f"{base_path}/projectphases/", "overwrite")


In [ ]:
df_projectphaes.show()

In [ ]:
#Read All dfs 
df_projectphaes = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/phases/*.parquet")
df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet")
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")
df_project_vitals = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project_vitals.parquet")
df_projectphaes = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/phases/*.parquet")

In [8]:
base_path = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/fv_raw_data"

df_project =readFromFile("parquet", f"{base_path}/project/")
df_intake =readFromFile("parquet", f"{base_path}/intkae/")
df_contact =readFromFile("parquet", f"{base_path}/contact/")
df_projecttype = readFromFile("parquet", f"{base_path}/projecttype/")
df_meds = readFromFile("parquet", f"{base_path}/meds/")
df_casesummary =readFromFile("parquet", f"{base_path}/casesummary/")
df_project_vitals =readFromFile("parquet", f"{base_path}/projectvitals/")
df_projectphases = readFromFile("parquet", f"{base_path}/projectphases")


In [ ]:
'''
base_path = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp/fv_raw_data"
writeToFile(df_project, f"{base_path}/project/", "overwrite")
writeToFile(df_intake, f"{base_path}/intkae/", "overwrite")
writeToFile(df_contact, f"{base_path}/contact/", "overwrite")
writeToFile(df_projecttype, f"{base_path}/projecttype/", "overwrite")
writeToFile(df_casesummary, f"{base_path}/casesummary/", "overwrite")
writeToFile(df_meds, f"{base_path}/meds/", "overwrite")
writeToFile(df_project_vitals, f"{base_path}/projectvitals/", "overwrite")
writeToFile(df_projectphaes, f"{base_path}/projectphases/", "overwrite")
'''


In [ ]:
df_meds.count()

In [9]:
df_project_vitals = df_project_vitals.withColumn("input_file", input_file_name())


In [18]:
df_meds_modified = df_meds.filter(df_meds.id.isNotNull())

In [ ]:
df_meds_modified.count()

In [10]:
df_project_vitals = df_project_vitals.withColumn('projectId', F.element_at(F.split(F.col('input_file'), '/'), -2))


In [11]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [ ]:
df_project_vitals.show()
#s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/10387917/project_vitals.parquet

In [12]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm.{}".format(table_name),
                            "dbtable": "tsm.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [13]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    #print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

Writing CMS_PeopleType
{"People_Type": "", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_ID": 1}
{"People_Type": "Adjuster", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_ID": 2}
{"People_Type": "Attorney", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_ID": 3}
{"People_Type": "Client", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_ID": 4}
{"People_Type": "Court", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "People_Sub_Type": null, "People_Type_ID": 5}
{"People_Type": "Defendant", "Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Custom1": null, "Custom2": null, "Custom3": null, "Pe

In [14]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    #print(result)
    #return result
    upload_result(result)
    
people_master_result = people_master_transformation(df_contact)

[TSMTableField(name='Truve_Org_ID', data_type='int', transform=TSMFieldTransform(source='internal', type='key', source_entity_type=None, source_entity_name=None, source_field=None)), TSMTableField(name='Client_Org_ID', data_type='string', transform=TSMFieldTransform(source='etl', type='org_id', source_entity_type=None, source_entity_name=None, source_field=None)), TSMTableField(name='People_ID', data_type='int', transform=TSMFieldTransform(source='etl', type='data', source_entity_type='core', source_entity_name='contact', source_field='personId')), TSMTableField(name='First_Name', data_type='string', transform=TSMFieldTransform(source='etl', type='data', source_entity_type='core', source_entity_name='contact', source_field='firstName')), TSMTableField(name='Middle_Name', data_type='string', transform=None), TSMTableField(name='Last_Name', data_type='string', transform=TSMFieldTransform(source='etl', type='data', source_entity_type='core', source_entity_name='contact', source_field='las

In [15]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

root
 |-- Truve_Org_ID: string (nullable = false)
 |-- Client_Org_ID: string (nullable = false)
 |-- Practice_Type_ID: integer (nullable = true)
 |-- Practice_Type_Name: string (nullable = true)
 |-- Custom1: string (nullable = true)
 |-- Custom2: string (nullable = true)
 |-- Custom3: string (nullable = true)

{'CMS_PracticeTypes': DataFrame[Truve_Org_ID: int, Client_Org_ID: string, Practice_Type_ID: int, Practice_Type_Name: string, Custom1: string, Custom2: string, Custom3: string]}
Writing CMS_PracticeTypes
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Practice_Type_Name": "ALI AWAD LAW TEMPLATE", "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 29908, "Practice_Type_Name": "Baby Formula / NEC cases", "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 30531, "Practice_Type_Name": "CEO Lawyer Summit", "Custom1": null, 

In [27]:
#Peoples Transformation
#df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    return result
    
phase_result = phases_transformation(df_projecttype)

+------------+-------------+----------------+--------+--------------------+-----------+--------------+------------------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Practice_Type_ID|Phase_ID|          Phase_Name|Phase_Order|Phase_Category|Phase_Sub_Category|Custom1|Custom2|Custom3|
+------------+-------------+----------------+--------+--------------------+-----------+--------------+------------------+-------+-------+-------+
|        6586|         6586|           18764|   91388|            New Lead|       null|          null|              null|   null|   null|   null|
|        6586|         6586|           18764|  103016|              Intake|       null|          null|              null|   null|   null|   null|
|        6586|         6586|           18764|   91389|         File Set-Up|       null|          null|              null|   null|   null|   null|
|        6586|         6586|           18764|   91390|           Treatment|       null|          null|              null|   

In [19]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    #print(result)
    upload_result(result)
    return result
    
result = value_transformation(df_meds_modified)

+------------+-------------+--------------+--------+--------------------+-------------------+-------------------+-----------+-------------+--------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|      Case_Figure_ID|        Figure_Type|    Figure_Sub_Type|Figure_Date|Figure_Status|   Value|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+--------------------+-------------------+-------------------+-----------+-------------+--------+-------+-------+-------+
|        6586|         6586|       9675645| 9675645|7d9f82d9-8dca-49b...|      Case Expenses|      Case Expenses|       null|         null|    35.0|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|3f9a8e3e-9454-45d...|      Case Expenses|      Case Expenses|       null|         null|    20.0|   null|   null|   null|
|        6586|         6586|       9675645| 9675645|93f98581-e6ab-4a0...|      Case Expenses|      Case Expenses|       null|         n

In [ ]:
#Peoples Transformation
'''
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    return result
    #print(result)
    #upload_result(result)
    
result = cases_transformation(df_project)
'''

In [20]:
#Peoples Transformation
def intake_transformation(df_intake, df_casesummary):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake, df_casesummary)
    return result 

result = intake_transformation(df_intake, df_casesummary)
#result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


+------------+-------------+--------------+--------+---------+---------------------------+--------------------+--------------+----------------+--------------+----------------+-----------------------+-------------------+-----------------+-----------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Intake_ID|Person_Performing_Intake_ID|       Intake_Source|Date_of_Intake|Date_of_Incident|Date_of_Signup|DUI_or_HitandRun|Referral_Fee_Percentage|If_Case_Referred_In|If_Qualified_Case|If_VIP_Lead|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+---------+---------------------------+--------------------+--------------+----------------+--------------+----------------+-----------------------+-------------------+-----------------+-----------+-------+-------+-------+
|        6586|         6586|       9252629| 9252629|  9252629|                   30625608|               Other|    2021-04-28|      2021-04-22|    2021-04-27|            None|   

In [ ]:
#Peoples Transformation
def casesummary_transformation(df_casesummary, df_project_vitals, df_project):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary, df_project_vitals, df_project)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary, df_project_vitals, df_project)

In [ ]:
processed_project_df = result["CMS_Cases"]
processed_project_df.printSchema()

In [21]:
#Peoples Transformation
def casetype_transformation(df_casesummary, project_df):
    #Read Raw Data
    result = builder.build_casetypes(df_casesummary, project_df)
    #print(result)
    #return result
    upload_result(result)
    return result
    
casetype_result = casetype_transformation(df_casesummary, df_project)

[StructField(caseType,StringType,true), StructField(Practice_Type_ID,IntegerType,true), StructField(Truve_Org_ID,StringType,false), StructField(Client_Org_ID,StringType,false), StructField(Case_Type_ID,IntegerType,true), StructField(Case_Type_Name,StringType,true)]
Writing CMS_CaseTypes
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 1, "Case_Type_Name": "County/Municipal Liability - Ante Litem", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 2, "Case_Type_Name": "Diminished Value (PD)", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": null, "Custom2": null, "Custom3": null}
{"Truve_Org_ID": 6586, "Client_Org_ID": "6586", "Practice_Type_ID": 18764, "Case_Type_ID": 3, "Case_Type_Name": "Dog Bite", "Case_Type_Category": null, "Case_Type_Sub_Category": null, "Custom1": nul

In [22]:
processed_case_type_df = casetype_result["CMS_CaseTypes"]

#Peoples Transformation
def cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary):
    #Read Raw Data
    result = builder.build_casemaster(df_project, processed_case_type_df, df_project_vitals, df_casesummary)
    #return result
    #print(result)
    upload_result(result)
    return result
    
result = cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary)

+--------------------+
|    Practice_Type_ID|
+--------------------+
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
|{'native': 18764,...|
+--------------------+
only showing top 20 rows

+--------------------+--------------------+-----------+--------------------+--------+-----+--------------------+-------------+
|           fieldName|        friendlyName|  fieldType|               value|position|links|          input_file|    projectId|
+--------------------+--------------------+-----------+--------------------+--------+-----+--------------------+-------------+
|paralegal223935Fu...|        Case Manage

In [23]:
processed_case_types = casetype_result["CMS_CaseTypes"]
#Peoples Transformation
def casesummary_transformation(df_casesummary, df_project_vitals, df_project, processed_case_types, df_intake):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary, df_project_vitals, df_project, processed_case_types, df_intake)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary, df_project_vitals, df_project, processed_case_types, df_intake)

+----------------+
|Practice_Type_ID|
+----------------+
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
+----------------+
only showing top 20 rows

+----------------+
|Practice_Type_ID|
+----------------+
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
|           18764|
+----------------+
only showing top 20 rows

{'CMS_CaseDetails': DataFrame[Case_I

In [29]:
#Peoples Transformation
def phasechanges_transformation(projectphases_df, processed_project_df, phases_df):
    #Read Raw Data
    result = builder.build_phasechanges(projectphases_df, processed_project_df, phases_df)
    print(result)
    upload_result(result)
    
phasechanges_transformation(projectphases_df=df_projectphases, 
                        processed_project_df=result["CMS_Cases"], 
                        phases_df=phase_result["CMS_Phases"])

+------------+-------------+--------------+--------+----------------+-----------------+--------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|Practice_Type_ID|Phase_Change_Date|Phase_ID|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+----------------+-----------------+--------+-------+-------+-------+
|        6586|         6586|       8827602| 8827602|           18764|       2022-10-19|  142845|   null|   null|   null|
|        6586|         6586|       8827602| 8827602|           18764|       2022-10-19|  163604|   null|   null|   null|
|        6586|         6586|      10849060|10849060|           18764|       2022-10-19|  163604|   null|   null|   null|
|        6586|         6586|      10849060|10849060|           18764|       2022-10-19|  163604|   null|   null|   null|
|        6586|         6586|      10849060|10849060|           18764|       2022-10-19|  142845|   null|   null|   null|
|        6586|         6586|    

In [ ]:
cms_cases_df = result["CMS_Cases"]

In [ ]:
cms_cases_df.select("Case_Type_ID").distinct().show()

In [ ]:
%stop_session

In [ ]:
df_casesummary.select("caseType").distinct().show(10)

In [ ]:
casetype_df = casetype_result["CMS_CaseTypes"]


In [ ]:
casetype_df.show()

In [ ]:
people_df = people_master_result["CMS_People"]

In [ ]:
people_df

In [ ]:
people_df.show()

In [ ]:

# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col,isnan,when,count
df2 = people_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in people_df.columns])
df2.show()


In [10]:
%stop_session

Stopping session: d5fcc2d4-b43d-4fdb-acc6-4f710f708a4b
Stopped session.


In [ ]:
case_df = result["CMS_Cases"]

In [ ]:
case_figure_df.select("Case_Figure_ID", "Case_ID").show()

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
df2 = case_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in case_df.columns])
df2.show()
